# Linearity in Finance

> To-do list:
* Examining the CAPM model with the efficient frontier and capital market line
* Solving for the security market line using regression
* Examining the APT model and performing a multivariate linear regression
* Understanding linear optimization in portfolio allocation
* Linear optimization using the PuLP package
* Understanding the outcomes of linear programming
* Introduction to integer programming
* Implementing a linear integer programming model with binary conditions
* Solving systems of linear equations with equalities using matrix linear algebra
* Solving systems of linear equations directly with the LU, Cholesky, and QR decomposition
* Solving systems of linear equations indirectly with the Jacobi and Gauss-Seidel method

## CAPM model

$$ R_i = R_f+\beta_i(R_{mkt} - R_f) $$

The `scipy.stats.linregress` function returns the slope of the regression line, the intercept of the regression line, the correlation coefficient, the p-value for a hypothesis test with null hypothesis of a zero slope, and the standard error of the estimate

| Time period | Stock returns | Market returns |
| ----------- | ----------- | ----------- |
| 1 | 0.065 | 0.055 |
|2 |0.0265|-0.09
|3|-0.0593|-0.041
|4|-0.001|0.045
|5|0.0346|0.022






In [3]:
""" Linear regression with SciPy """
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)

print(beta, alpha)

0.5077431878770808 -0.008481900352462384


* The equation describing the SML can be written as:
$$ E(R_i)=R_f +β_i [E(R_M)−R_f] $$

## The Arbitrage Pricing Theory (APT) model

$$ E[R_i]=α_i+β_{i,1}F_1+β_{i,2}F_2+...+β_{i,j}F_j $$

In [4]:
""" Least squares regression with statsmodels """
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8)
                        for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0]  # First column values as Y
x_values = all_values[:, 1:]  # All other values as X
x_values = sm.add_constant(x_values)  # Include the intercept
results = sm.OLS(y_values, x_values).fit()  # Regress and fit the model

In [5]:
results.summary()


/Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                 -2.724
Method:                 Least Squares   F-statistic:                    0.1640
Date:                Tue, 06 Jun 2023   Prob (F-statistic):              0.957
Time:                        15:56:03   Log-Likelihood:                 6.7045
No. Observations:                   9   AIC:                             2.591
Df Residuals:                       1   BIC:                             4.169
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7998      2.580      0.310      0.809     -31.977      33.576
x1             0.0448      1.960      0.023      0.985     -24.856      24.946
x2             0.0010      2.997      0.000      1.000     -38.075      38.077
x3            -0.2927      0.602     -0.486      0.712      -7.944       7.359
x4             0.1684      3.126      0.054      0.966     -39.547      39.884
x5             0.0351      0.849      0.041      0.974     -10.746      10.817
x6             0.1171      0.483      0.242      0.849      -6.024       6.258
x7            -0.4595      2.339     -0.196      0.877     -30.183      29.264
==============================================================================
Omnibus:                        1.910   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.385   Jarque-Bera (JB):                0.863
Skew:                           0.740   Prob(JB):                        0.649
Kurtosis:                       2.669   Cond. No.                         91.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""